In [1]:
import torch
sentence = torch.tensor([0,7,1,2,5,6,4,3]) # can you help me to translate this sentence
sentence

tensor([0, 7, 1, 2, 5, 6, 4, 3])

Let's assume that we already encoded this sentence into a real number vector representations via an embedding layer. Here Our emedding size is 16, and we assume that the dictionary size is 10, The following code will produce the word embeddings ofr our eight words.

In [2]:
torch.manual_seed(123)
embed = torch.nn.Embedding(10,16)
embedded_sentence = embed(sentence).detach()
embedded_sentence

tensor([[ 3.3737e-01, -1.7778e-01, -3.0353e-01, -5.8801e-01,  3.4861e-01,
          6.6034e-01, -2.1964e-01, -3.7917e-01,  7.6711e-01, -1.1925e+00,
          6.9835e-01, -1.4097e+00,  1.7938e-01,  1.8951e+00,  4.9545e-01,
          2.6920e-01],
        [-9.4053e-01, -4.6806e-01,  1.0322e+00, -2.8300e-01,  4.9275e-01,
         -1.4078e-02, -2.7466e-01, -7.6409e-01,  1.3966e+00, -9.9491e-01,
         -1.5822e-03,  1.2471e+00, -7.7105e-02,  1.2774e+00, -1.4596e+00,
         -2.1595e+00],
        [-7.7020e-02, -1.0205e+00, -1.6896e-01,  9.1776e-01,  1.5810e+00,
          1.3010e+00,  1.2753e+00, -2.0095e-01,  4.9647e-01, -1.5723e+00,
          9.6657e-01, -1.1481e+00, -1.1589e+00,  3.2547e-01, -6.3151e-01,
         -2.8400e+00],
        [-1.3250e+00,  1.7843e-01, -2.1338e+00,  1.0524e+00, -3.8848e-01,
         -9.3435e-01, -4.9914e-01, -1.0867e+00,  8.8054e-01,  1.5542e+00,
          6.2662e-01, -1.7549e-01,  9.8284e-02, -9.3507e-02,  2.6621e-01,
         -5.8504e-01],
        [ 2.5529e-01

In [3]:
embedded_sentence.shape

torch.Size([8, 16])

In [4]:
omega = torch.empty(8,8)
for i, xi in enumerate(embedded_sentence):
	for j, xj in enumerate(embedded_sentence):
		omega[i, j] = torch.dot(xi, xj)

In [5]:
omega

tensor([[ 9.7601,  1.7326,  4.7543, -1.3587,  0.4752, -1.6717,  1.0227, -0.1286],
        [ 1.7326, 16.0787,  9.0642, -0.3370,  1.1368,  1.1972,  1.6485, -1.2789],
        [ 4.7543,  9.0642, 22.6615, -0.8519,  7.7799,  2.7483, -0.6832,  1.6236],
        [-1.3587, -0.3370, -0.8519, 13.9473, -1.4198, 10.9659, -0.5887,  2.3869],
        [ 0.4752,  1.1368,  7.7799, -1.4198, 13.7511, -6.8568, -2.5114, -3.3468],
        [-1.6717,  1.1972,  2.7483, 10.9659, -6.8568, 24.6738, -3.8294,  4.9581],
        [ 1.0227,  1.6485, -0.6832, -0.5887, -2.5114, -3.8294, 15.8691,  2.0269],
        [-0.1286, -1.2789,  1.6236,  2.3869, -3.3468,  4.9581,  2.0269, 18.7382]])

In [6]:
omega_mat = embedded_sentence.matmul(embedded_sentence.T)
omega_mat

tensor([[ 9.7601,  1.7326,  4.7543, -1.3587,  0.4752, -1.6717,  1.0227, -0.1286],
        [ 1.7326, 16.0787,  9.0642, -0.3370,  1.1368,  1.1972,  1.6485, -1.2789],
        [ 4.7543,  9.0642, 22.6615, -0.8519,  7.7799,  2.7483, -0.6832,  1.6236],
        [-1.3587, -0.3370, -0.8519, 13.9473, -1.4198, 10.9659, -0.5887,  2.3869],
        [ 0.4752,  1.1368,  7.7799, -1.4198, 13.7511, -6.8568, -2.5114, -3.3468],
        [-1.6717,  1.1972,  2.7483, 10.9659, -6.8568, 24.6738, -3.8294,  4.9581],
        [ 1.0227,  1.6485, -0.6832, -0.5887, -2.5114, -3.8294, 15.8691,  2.0269],
        [-0.1286, -1.2789,  1.6236,  2.3869, -3.3468,  4.9581,  2.0269, 18.7382]])

In [7]:
# Using torch.allclose() function to check that this matrix multiplication produces the expected results.
# If two tensors contain the same values, torch.allclose() returns True.

torch.allclose(omega_mat, omega)


True

In [8]:
# Calculating the attention weights using pytorch softmax function
import torch.nn.functional as F
attention_weights = F.softmax(omega, dim=1)
attention_weights.shape

torch.Size([8, 8])

In [9]:
attention_weights.sum(dim=1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])

### Summarize the three main steps behind the self attention operation

1. For a given input element xi and each jth element in the set {1, ..., T}, Compute the dot product, xi'.xj
2. Obtain the attention weight, $\alpha i,j$ by normalizing the dot products using the softmax function
3. Compute the output, zi, as the weighted sum over the entire input sequence

In [10]:
# lets compute the context vector for the second input wordk, i.e. z2
x2 = embedded_sentence[1,:]
context_vec2 = torch.zeros(x2.shape)
for j in range(8):
	xj = embedded_sentence[j,:]
	context_vec2 += attention_weights[1,j] * xj
context_vec2

tensor([-9.3975e-01, -4.6856e-01,  1.0311e+00, -2.8192e-01,  4.9373e-01,
        -1.2896e-02, -2.7327e-01, -7.6358e-01,  1.3958e+00, -9.9543e-01,
        -7.1287e-04,  1.2449e+00, -7.8077e-02,  1.2765e+00, -1.4589e+00,
        -2.1601e+00])

In [11]:
# we can acheive this more efficiently by using matrix multiplication.  
context_vectors = torch.matmul(attention_weights, embedded_sentence)

In [12]:
torch.allclose(context_vec2, context_vectors[1])

True

## parameterizing the self-attention mechanism: Scaled dot-product attention
Note that in the previous section we don't involve any learnable parameters when computing the outputs. To make the self attention mechanism more flexible and amenable to model optimization, we will introduce three additional weight metrics that can be fit as model parameters during model training. we denote those three weight metrices as Uq, Uk and Uv. They are used to project the inputs into query, key and value sequence elements

In [13]:
torch.manual_seed(123)
d = embedded_sentence.shape[1]
U_query = torch.rand(d,d)
U_key =  torch.rand(d,d)
U_value = torch.rand(d,d)

In [14]:
x2 = embedded_sentence[1]
query2 = U_query.matmul(x2)

In [15]:
key2 = U_key.matmul(x2)
value2 = U_value.matmul(x2)

In [16]:
keys = U_key.matmul(embedded_sentence.T).T
torch.allclose(key2, keys[1])
values = U_value.matmul(embedded_sentence.T).T
torch.allclose(value2, values[1])

True

In [17]:
omega_23 = query2.dot(keys[2])
omega_23

tensor(14.3667)

In [19]:
omega_2 = query2.matmul(keys.T)
omega_2

tensor([-25.1623,   9.3602,  14.3667,  32.1482,  53.8976,  46.6626,  -1.2131,
        -32.9392])

In [20]:
attention_weights_2 = F.softmax(omega_2 / d**0.5, dim=0)
attention_weights_2

tensor([2.2317e-09, 1.2499e-05, 4.3696e-05, 3.7242e-03, 8.5596e-01, 1.4026e-01,
        8.8897e-07, 3.1935e-10])

In [21]:
context_vector_2 = attention_weights_2.matmul(values)
context_vector_2

tensor([-1.2226, -3.4387, -4.3928, -5.2125, -1.1249, -3.3041, -1.4316, -3.2765,
        -2.5114, -2.6105, -1.5793, -2.8433, -2.4142, -0.3998, -1.9917, -3.3499])

## Encoding context embeddings via multi-head attention

In [22]:
torch.manual_seed(123)
d = embedded_sentence.shape[1]
one_U_query = torch.rand(d,d)


In [23]:
h=8
multihead_U_query = torch.rand(h,d,d)
multihead_U_key = torch.rand(h,d,d)
multihead_U_value = torch.rand(h,d,d)


In [25]:
multihead_query_2 = multihead_U_query.matmul(x2)
multihead_query_2.shape
torch.Size([8,16])


torch.Size([8, 16])

In [26]:
multihead_key_2 = multihead_U_key.matmul(x2)
multihead_value_2 = multihead_U_value.matmul(x2)
multihead_key_2[2]

tensor([-1.9619, -0.7701, -0.7280, -1.6840, -1.0801, -1.6778,  0.6763,  0.6547,
         1.4445, -2.7016, -1.1364, -1.1204, -2.4430, -0.5982, -0.8292, -1.4401])

In [27]:
stacked_inputs = embedded_sentence.T.repeat(8,1,1)
stacked_inputs.shape

torch.Size([8, 16, 8])

In [28]:
multihead_keys = torch.bmm(multihead_U_key, stacked_inputs) # bmm-> batch matrix multiplication
multihead_keys.shape

torch.Size([8, 16, 8])

In [30]:
multihead_keys = multihead_keys.permute(0,2,1)
multihead_keys.shape

torch.Size([8, 16, 8])

In [31]:
multihead_keys[2,1]

tensor([ 0.2835, -0.7701,  0.4757, -1.1125, -1.8784, -1.8276, -1.9170,  3.1078])

In [32]:
multihead_values = torch.matmul(
	multihead_U_value, stacked_inputs
)
multihead_values = multihead_values.permute(0,2,1)


In [35]:
multihead_z_2 = torch.rand(8,16)

In [37]:
linear = torch.nn.Linear(8*16, 16)
context_vector_2 = linear(multihead_z_2.flatten())
context_vector_2.shape

torch.Size([16])

## Using GPT-2 to generate new text

In [1]:
from transformers import pipeline, set_seed
generator = pipeline("text-generation", model="gpt2")
set_seed(123)
generator("Hey readers..., Today is ", max_length = 20, num_return_sequences=3)


/home/santosh/ML-DL_codeBook/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'generated_text': 'Hey readers..., Today is \xa0the day to take a quick glance at how much can vary'},
 {'generated_text': 'Hey readers..., Today is \xa0the 8th of October!\xa0 We still have an\xa0'},
 {'generated_text': "Hey readers..., Today is 『Rune of Twilight』. I didn't just come to"}]